In [ ]:
# import libraries
import numpy as np
import pandas as pd
import GPy
import sys
sys.path.append('../')
import BayesianOptimization

In [ ]:
# Mixed integer problem
# Objective function
# -(5*y1 + 8*y2 + 6*y3 + 10*y4 + 6*y5 - 10*x1 - 15*x2 - 15*x5 + 15*x3 + 5*x4 - 20*x6 + exp(x1) + exp(x2/1.2) - 60*log(x3 + x4 + 1) + 140)
# Constraints
# y1 + y2 == 1
# -log(x3 + x4 + 1) <= 0
# -x1 - x2 - 2*x5 + x3 + 2*x6 <= 0
# -x1 - x2 - 0.75*x5 + x3 + 2*x6 <= 0
# x5 - x6 <= 0
# 2*x5 - x3 - 2*x6 <= 0
# -0.5*x3 + x4 <= 0
# 0.2*x3 - x4 <= 0
# exp(x1) - U*y1 <= 1
# exp(x2/1.2) - U*y2 <= 1
# 1.25*x5 - U*y3 <= 0
# x3 + x4 - U*y4 <= 0
# -2*x5 + 2*x6 - U*y5 <= 0
# y4 + y5 <= 1
# Parameters
# U = 10

# Best solution
# y = [0, 1, 1, 1, 0]
# x = [0, 2, 0.65, 0.32, 1.07, 1.07]
# f(x, y) = 73

# Number of continuous variables (n)
n = 6
# Number of discrete variables (m)
m = 5
# Number of variables (n + m)
n_tot = n+m
# Number of equality constraints (r)
r = 1
# Number of inequality constraints (s)
s = 13
# Parameter U
U = 10

# Define the objective function
def f(x, y):
    return -(5*y[:,0] + 8*y[:,1] + 6*y[:,2] + 10*y[:,3] + 6*y[:,4] - 10*x[:,0] - 15*x[:,1] - 15*x[:,4] + 15*x[:,2] + 5*x[:,3] - 20*x[:,5] + np.exp(x[:,0]) + np.exp(x[:,1]/1.2) - 60*np.log(x[:,2] + x[:,3] + 1) + 140)

# Define the constraints
# equality constraints
def g(x, y):
    G = np.zeros((y.shape[0], r))
    G[:,0] = y[:,0] + y[:,1] - 1
    return G

# inequality constraints
def h(x, y):
    H = np.zeros((y.shape[0], s))
    H[:,0] = -np.log(x[:,2] + x[:,3] + 1)
    H[:,1] = -x[:,0] - x[:,1] - 2*x[:,4] + x[:,2] + 2*x[:,5]
    H[:,2] = -x[:,0] - x[:,1] - 0.75*x[:,4] + x[:,2] + 2*x[:,5]
    H[:,3] = x[:,4] - x[:,5]
    H[:,4] = 2*x[:,4] - x[:,2] - 2*x[:,5]
    H[:,5] = -0.5*x[:,2] + x[:,3]
    H[:,6] = 0.2*x[:,2] - x[:,3]
    H[:,7] = np.exp(x[:,0]) - U*y[:,0] - 1
    H[:,8] = np.exp(x[:,1]/1.2) - U*y[:,1] - 1
    H[:,9] = 1.25*x[:,4] - U*y[:,2]
    H[:,10] = x[:,2] + x[:,3] - U*y[:,3]
    H[:,11] = -2*x[:,4] + 2*x[:,5] - U*y[:,4]
    H[:,12] = y[:,3] + y[:,4] - 1
    return H

# Bounds
x_l = np.zeros(n)
x_u = np.array([2, 2, 2, 3, 3, 3])
y_l = np.zeros(m, dtype=int)
y_u = np.ones(m, dtype=int)
# Bounds
bounds_x = np.array([x_l, x_u])
bounds_y = np.array([y_l, y_u])

In [ ]:
x_opt = np.array([0, 2, 0.65, 0.32, 1.07, 1.07]).reshape(1, -1)
y_opt = np.array([0, 1, 1, 1, 0]).reshape(1, -1)

In [ ]:
# Load initial data
D_0 = np.loadtxt("initial_data_ex2.csv", delimiter=",")

# Select surrogate model, AF, kernel and sampling strategy
surrogate = 'SGP'
kernel = GPy.kern.RBF(input_dim=n_tot, variance=1.)
adquisition = "EI"
sample_y = "Uniform"
sample_x = "Uniform"
param_est_Bernoulli = "None"
param_est_Gaussian = "None"

In [ ]:
labels_D = ["x1", "x2", "x3", "x4", "x5", "x6", "y1", "y2", "y3", "y4", "y5", "Omega"]
labels_D_best = ["x1", "x2", "x3", "x4", "x5", "x6", "y1", "y2", "y3", "y4", "y5", "Omega", "p", "f"]
points = 100000
for i in range(5):
    # Run the optimization 10 times, save the solutions with tuples
    bo = BayesianOptimization.BO_GPy(f, g, h, None, None, None,
                                    bounds_x, bounds_y, kernel, n, m, 
                                    points = points, t_max = 500, j_max = 50,
                                    surrogate = surrogate, adquisition = adquisition, 
                                    sample_y = sample_y, sample_x = sample_x, param_est_bernoulli = param_est_Bernoulli, param_est_gaussian = param_est_Gaussian,
                                    D_0 = D_0, early_stop=False, DomainReduction=False, verbose=True)
    D, D_best = bo.fit()
    #df_D = pd.DataFrame(D, columns=labels_D)
    #df_D_best = pd.DataFrame(D_best, columns=labels_D_best)
    #df_D.to_csv("Example_2_final" + ".csv", index=False, mode="a")
    #df_D_best.to_csv("Example_2_best_final" + ".csv", index=False, mode="a")